In [1]:
!pip install bs4         #beautifulsoup 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=f377f6173c787ec5a62e8a8d51cdc5ead7dec2570aefbe41072fbeee6fb3bf16
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [2]:
from google.colab import drive          #구글드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd      

In [4]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [60]:
#한 페이지에 대해 스크래핑
import requests
from bs4 import BeautifulSoup

url = "https://kin.naver.com/qna/list.naver?dirId=70112"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

titles = soup.select("td.title a")
fields = soup.find_all("td", class_="field")

column1 = []
column2 = []
for title, field in zip(titles, fields):
    column1.append(title.text)
    column2.append(field.text)

df1 = pd.DataFrame({'Title': column1, 'Field': column2})
df = pd.concat([df,df1])


In [73]:
#페이지를 넘겨가며 동적으로 스크래핑
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode

base_url = "https://kin.naver.com/qna/list.naver"
query_params = {
    "dirId": "70114",
    "queryTime": "2023-05-15 09:46:52",
}
column1 = []
column2 = []
page = 1

while (page<30) : 
    query_params["page"] = page
    url = base_url + "?" + urlencode(query_params)

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    titles = soup.select("td.title a")
    fields = soup.find_all("td", class_="field")

    if not titles or not fields:
        break

    for title, field in zip(titles, fields):
        column1.append(title.text)
        column2.append(field.text)

    page += 1

df1 = pd.DataFrame({'Title': column1, 'Field': column2})
df = pd.concat([df,df1])


In [74]:
len(df)

13102

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
df.to_csv('/content/drive/MyDrive/data.csv')

In [38]:
df[0:-10]

,Unnamed: 0,Title,Field
0,0.0,발목 인대,정형외과
1,1.0,스마일 라식 2일 전 음주,안과
2,2.0,왼쪽 허벅지 안쪽아파요,정형외과
3,3.0,실수 잘못을 할때마다 너무 ㅈ고싶어요...,정신건강의학과
4,4.0,키분포도 궁금점,소아청소년과
...,...,...,...
3,NaN,도파민에 대해서 질문있습니다!!,신경과
4,NaN,보톡스 기립경사 검사,신경과
5,NaN,"국민들이 화장실에서 구토,기절하는이유는, 보건복지부의 외...",신경과
6,NaN,데이터 평균,신경과
